<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

     |████████████████████████████████| 1.5MB 15.3MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 85 kB/s 
     |████████████████████████████████| 1.1 MB 64.9 MB/s 
     |████████████████████████████████| 9.1 MB 34.4 MB/s 
     |████████████████████████████████| 456 kB 63.1 MB/s 
     |████████████████████████████████| 113 kB 59.8 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=df3584b4ca612314e1289644087685c4ad2bea0514e8d4f89b2b50c720284c8e
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: catalogue
    Found existing installati

In [2]:
pip install pymorphy2[fast]

     |████████████████████████████████| 371 kB 21.5 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=865707 sha256=c4635cb5774d4213a539cec9e75a39bb67b59f807b2caf3e7d41d0a60802a5cc
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
pip install Gensim

In [4]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd

In [5]:
nlp = spacy.load('ru_core_news_md')

In [9]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt

--2021-04-23 20:35:06--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3638613 (3.5M) [text/plain]
Saving to: ‘neuroleptic_ru.txt’

neuroleptic_ru.txt  100%[===================>]   3.47M  --.-KB/s    in 0.05s   

2021-04-23 20:35:06 (69.0 MB/s) - ‘neuroleptic_ru.txt’ saved [3638613/3638613]



In [6]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru.txt

--2021-04-23 20:34:30--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12963713 (12M) [text/plain]
Saving to: ‘rakpobedim_ru.txt’

rakpobedim_ru.txt   100%[===================>]  12.36M  --.-KB/s    in 0.1s    

2021-04-23 20:34:31 (117 MB/s) - ‘rakpobedim_ru.txt’ saved [12963713/12963713]



In [7]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit.txt

--2021-04-23 20:34:34--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6298252 (6.0M) [text/plain]
Saving to: ‘hv-info_gepatit.txt’

hv-info_gepatit.txt 100%[===================>]   6.01M  --.-KB/s    in 0.08s   

2021-04-23 20:34:34 (71.7 MB/s) - ‘hv-info_gepatit.txt’ saved [6298252/6298252]



In [ ]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-04-11 11:52:41--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.1’

news_upos_cbow_300_ 100%[===================>] 133.81M   189MB/s    in 0.7s    

2021-04-11 11:52:42 (189 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.1’ saved [140310878/140310878]



In [10]:
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()


In [16]:
handler = open('hv-info_gepatit.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [17]:
handler = open('rakpobedim_ru.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [18]:
len(my_txt)

13016911

In [22]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-04-23 20:38:21--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz’

news_upos_cbow_300_ 100%[===================>] 133.81M  32.6MB/s    in 4.7s    

2021-04-23 20:38:27 (28.2 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz’ saved [140310878/140310878]



In [19]:
posts = str_list = my_txt.split('----')

In [20]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [23]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [24]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized)
total_examples = model.corpus_count
model.build_vocab([list(w2v.vocab.keys())], update=True)
model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(8406168, 9683300)

In [25]:
model.most_similar('препарат')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('нельзя', 0.7883257865905762),
 ('который', 0.7685508728027344),
 ('его', 0.7212514877319336),
 ('человек', 0.7043541073799133),
 ('счёт', 0.6969262361526489),
 ('нейролептик', 0.6952941417694092),
 ('нам', 0.6857447624206543),
 ('дольше', 0.677767813205719),
 ('постоянно', 0.6756583452224731),
 ('мощный', 0.6753535270690918)]

In [26]:
# model['алимемазин']
model.wv.most_similar('алимемазин')

[('чистили', 0.795113205909729),
 ('блоками', 0.7911574840545654),
 ('выравнялся', 0.7900457382202148),
 ('пребывал', 0.774469256401062),
 ('допиваю', 0.7743908166885376),
 ('пропадают', 0.76788330078125),
 ('расти', 0.7603928446769714),
 ('третичная', 0.7532668709754944),
 ('слабоумным', 0.7530084252357483),
 ('сбор', 0.7485941648483276)]

In [27]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('азалептин')))[0][0]

0.2850779

In [28]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('стул')))[0][0]

0.19343388

In [30]:
# Модель есть, теперь надо набрать корпус из трех форумов примерно поровну, и чтобы не помереть при разметке.

In [41]:
short_txt = " "
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
short_txt = handler.read()[:200000]
handler.close()

handler = open('rakpobedim_ru.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:200000]
handler.close()

handler = open('hv-info_gepatit.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:200000]
handler.close()

In [42]:
len(short_txt)

600000

In [43]:
short_posts = str_list = short_txt.split('----')

In [44]:
len(short_posts)

1615

In [45]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in short_posts]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [46]:
doc = nlp(u'Кто принимал его в своей практике? Это не хороший препарат.')

In [47]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [95]:
sample = 'их'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)


Одуш None
Падеж accs
Пол None
Число plur
Лицо 3per
Часть речи NPRO
Время None
Залог None


In [48]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag)


Кто nsubj принимал VERB [] NPRO,masc sing,nomn
принимал ROOT принимал VERB [Кто, его, практике, ?] VERB,impf,tran masc,sing,past,indc
его obj принимал VERB [] NPRO,masc,3per,Anph sing,accs
в case практике NOUN [] PREP
своей det практике NOUN [] ADJF,Apro,Anph femn,sing,gent
практике obl принимал VERB [в, своей] NOUN,inan,femn sing,loct
? punct принимал VERB [] PNCT
Это nsubj препарат NOUN [] PRCL
не advmod хороший ADJ [] PRCL
хороший amod препарат NOUN [не] ADJF,Qual masc,sing,nomn
препарат ROOT препарат NOUN [Это, хороший, .] NOUN,inan,masc sing,nomn
. punct препарат NOUN [] PNCT


In [49]:
def find_args(my_text, my_spacy):
  sents = text.split('.')
  for sent in sents:
    doc = my_spacy(u'Кто принимал его в своей практике')

In [50]:
token_list = [i for i in doc]

In [51]:
for token in doc:
  if token.dep_ == 'obj':
    print(token.text, token.dep_, token.head.text, token.head.pos_,
        [child for child in token.children])

его obj принимал VERB []


In [52]:
sentences_tokenized[2]

['в',
 'волгограде',
 'есть',
 'почти',
 'во',
 'всех',
 'аптеках',
 'стоит',
 '80',
 'руб',
 'мы',
 'его',
 'применяемм',
 'в',
 'основном',
 'как',
 'вегетостабилизатор']

In [53]:
sentences_for_annts = [' '.join(sent) for sent in sentences_tokenized]
sentences_for_annts = [sent.split() for sent in set(sentences_for_annts)]
sents_for_annts= []
for sent in sentences_for_annts:
  sent.append('\n')
  sents_for_annts.extend(sent) 

In [54]:
len(sents_for_annts)

85522

In [55]:
df = pd.DataFrame(columns=['TOKEN', 'NER', 'COREFERENCE_CLUSTER'])

In [56]:
df['TOKEN'] = sents_for_annts
df['NER'] = [0 for i in range(len(sents_for_annts))]

In [57]:
df

,TOKEN,NER,COREFERENCE_CLUSTER
0,\n,0,NaN
1,мне,0,NaN
2,назначили,0,NaN
3,хлорпротиксен,0,NaN
4,вместо,0,NaN
...,...,...,...
85517,информацию,0,NaN
85518,ценно,0,NaN
85519,и,0,NaN
85520,ободряет,0,NaN


In [58]:
df.to_csv('for_annts3k.csv', sep=' ', index=False)

In [59]:
from google.colab import files


In [61]:
files.download('for_annts3k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

***Разметка для кореференции***

In [73]:
# sents_for_annts[1]
sentences_for_annts[8]

['snay',
 'сказал',
 'а',
 '24',
 '9',
 '2008',
 '16',
 '59',
 'а',
 'есть',
 'ли',
 'какой',
 'нибудь',
 'способ',
 'проверить',
 'что',
 'то',
 'вроде',
 'анализа',
 'сколько',
 'по',
 'времени',
 'нужно',
 'принимать',
 'этот',
 'препарат',
 'как',
 'мне',
 'объяснила',
 'мой',
 'врач',
 'нужно',
 'ждать',
 'пока',
 'не',
 'помрёт',
 'тот',
 'самый',
 'ген',
 'который',
 'запустил',
 'болезнь',
 'или',
 'пока',
 'организм',
 'не',
 'привыкнет',
 'к',
 'такому',
 'состоянию',
 'и',
 'ему',
 'уже',
 'не',
 'нужны',
 'будут',
 'таблетки',
 'но',
 'это',
 'так',
 'расплывчато',
 'и',
 'неопределенно',
 'что',
 'вводит',
 'в',
 'замешательство',
 'должны',
 'же',
 'быть',
 'какие',
 'то',
 'конкретные',
 'признаки',
 'как',
 'работает',
 'мозг',
 'нужен',
 'ли',
 'ему',
 'ещё',
 'препарат',
 'как',
 'себя',
 'будет',
 'вести',
 'мозг',
 'после',
 'отмены',
 'препарата',
 'и',
 'как',
 'он',
 'себя',
 'ведет',
 'во',
 'время',
 'его',
 'приема',
 'я',
 'понимаю',
 'что',
 'область',
 'пси